In [2]:
##   MLP: https://www.kaggle.com/rgaddati/santander-customer-satisfaction/santander1

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#from __future__ import division

%matplotlib inline  
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedKFold 
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import describe
from sklearn.preprocessing import StandardScaler, scale
from sklearn.grid_search import GridSearchCV

/home/sirorezka/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/sirorezka/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
df_train = pd.read_csv("Data/train.csv")
df_test = pd.read_csv("Data/test.csv")

y_train = df_train['TARGET'].values
id_test = df_test['ID']

In [4]:
print id_test.shape
print y_train.shape

(75818,)
(76020,)


In [5]:
df_train = pd.read_csv("Data/proc_train.csv", header=None)
df_test = pd.read_csv("Data/proc_test.csv", header=None)

print(df_test.shape)
print(df_train.shape)

(75818, 355)
(76020, 355)


In [6]:
X_train  = df_train.values
X_test = df_test.values

In [7]:
# classifier
clf = xgb.XGBClassifier(missing=np.nan, gamma=0, reg_lambda=4, max_depth=5, n_estimators=500, learning_rate=0.02, nthread=8, subsample=0.95, colsample_bytree=0.85, seed=4242)

X_fit, X_eval, y_fit, y_eval = train_test_split(X_train, y_train, test_size=0.3, random_state = 44)

# fitting
#clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])

clf.fit(X_fit, y_fit, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])

print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]), roc_auc_score(y_eval, clf.predict_proba(X_eval)[:,1]))

('Overall AUC:', 0.8646430197353161, 0.84190924353551033)


In [34]:
xgb_model.get_params()

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.85,
 'gamma': 0,
 'learning_rate': 0.02,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 500,
 'nthread': 8,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 2,
 'scale_pos_weight': 1,
 'seed': 4242,
 'silent': True,
 'subsample': 0.95}

In [45]:
##print(clf.best_score_)
print(clf.best_params_)

{'n_estimators': 400, 'reg_lambda': 4, 'max_depth': 5}


In [37]:
xgb_model = xgb.XGBClassifier(missing=np.nan, gamma=0, reg_lambda=2, max_depth=5, n_estimators=500, learning_rate=0.02, nthread=8, subsample=0.95, colsample_bytree=0.85, seed=4242)

clf = GridSearchCV(xgb_model,
                   {'max_depth': [4,5,6,7],
                    'n_estimators': [300,400,500,600],
                    'reg_lambda': [0.5,1,2,4,5,7,10,15,20,25]
                    }, verbose=5, scoring = "roc_auc", n_jobs=8)

clf.fit(X_fit, y_fit)

print(clf.best_score_)
print(clf.best_params_)

## {'n_estimators': 400, 'reg_lambda': 4, 'max_depth': 5}

Fitting 3 folds for each of 160 candidates, totalling 480 fits


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  2.8min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed: 24.7min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 79.2min
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed: 165.7min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed: 301.5min
[Parallel(n_jobs=8)]: Done 480 out of 480 | elapsed: 351.6min finished


0.837276865944
{'n_estimators': 400, 'reg_lambda': 4, 'max_depth': 5}
[CV] n_estimators=300, reg_lambda=0.5, max_depth=4 ...................
[CV] n_estimators=300, reg_lambda=0.5, max_depth=4 ...................
[CV] n_estimators=300, reg_lambda=0.5, max_depth=4 ...................
[CV] n_estimators=300, reg_lambda=1, max_depth=4 .....................
[CV] n_estimators=300, reg_lambda=1, max_depth=4 .....................
[CV] n_estimators=300, reg_lambda=1, max_depth=4 .....................
[CV] n_estimators=300, reg_lambda=2, max_depth=4 .....................
[CV] n_estimators=300, reg_lambda=2, max_depth=4 .....................
[CV]  n_estimators=300, reg_lambda=0.5, max_depth=4, score=0.830146 - 3.0min[CV]  n_estimators=300, reg_lambda=0.5, max_depth=4, score=0.842720 - 2.8min[CV]  n_estimators=300, reg_lambda=0.5, max_depth=4, score=0.834677 - 3.0min[CV]  n_estimators=300, reg_lambda=1, max_depth=4, score=0.830158 - 2.7min[CV]  n_estimators=300, reg_lambda=1, max_depth=4, score=0.8

In [35]:
GridSearchCV??

In [18]:
xgb.XGBClassifier??

In [22]:
# predicting
y_pred= clf.predict_proba(X_test)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("predictions/submission_xgb.csv", index=False)

print('Completed!')

Completed!


In [11]:
## Learning with cross_validation

skf = StratifiedKFold (y_train,n_folds=5, random_state = 48)

rc_eval_dt = []
for fit_index, test_index in skf:
        # print("TRAIN:", fit_index, "TEST:", test_index)
        X_fit, X_eval = X_train[fit_index], X_train[test_index]
        y_fit, y_eval = y_train[fit_index], y_train[test_index]
        
        clf.fit(X_fit, y_fit, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])
        
        rc_fit  = roc_auc_score(y_fit, clf.predict_proba(X_fit)[:,1])
        rc_eval = roc_auc_score(y_eval, clf.predict_proba(X_eval)[:,1])
        rc_train  = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
        
        rc_eval_dt.append(rc_eval)
        print (rc_fit,rc_eval,rc_train)

rc_eval_dt = np.array(rc_eval_dt)
print ("\n")
print ("results: ", rc_eval_dt.min(),rc_eval_dt.max(),rc_eval_dt.mean())

(0.82925183744912734, 0.81684513695019678, 0.82676223764149526)
(0.8387333682605963, 0.81403982661370033, 0.83378117410530594)
(0.85722421034670082, 0.842733223637958, 0.85432520938338907)
(0.85429906580611092, 0.8525390044123603, 0.85394502559547192)
(0.85567309707810935, 0.83600849244319786, 0.8517707345870571)


('results: ', 0.81403982661370033, 0.8525390044123603, 0.83243313681148268)


In [12]:
d_var = [50,75,100,125,150,175,200,225,250,275,300,325]

for d in d_var:
    skf = StratifiedKFold (y_train,n_folds=5, random_state = 48)

    rc_eval_dt = []
    for fit_index, test_index in skf:

            # print("TRAIN:", fit_index, "TEST:", test_index)
            X_fit, X_eval = X_train[fit_index][:,feature_selected[0:d]], X_train[test_index][:,feature_selected[0:d]]
            y_fit, y_eval = y_train[fit_index], y_train[test_index]
            
            clf = xgb.XGBClassifier(missing=np.nan, max_depth=3, n_estimators=550, learning_rate=0.03, nthread=8, subsample=0.95, colsample_bytree=0.85, seed=4242)
            clf.fit(X_fit, y_fit, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])    
            
            rc_fit  = roc_auc_score(y_fit, clf.predict_proba(X_fit)[:,1])
            rc_eval = roc_auc_score(y_eval, clf.predict_proba(X_eval)[:,1])
            rc_train  = roc_auc_score(y_train, clf.predict_proba(X_train[:,feature_selected[0:d]])[:,1])
            rc_eval_dt.append(rc_eval)
            
            print (rc_fit,rc_eval,rc_train)

    rc_eval_dt = np.array(rc_eval_dt)

    #print ("\n")
    print ("results: ",d," -- ",rc_eval_dt.min(),rc_eval_dt.max(),rc_eval_dt.mean())

(0.82581993002108967, 0.81269680625857832, 0.82319398494389151)
(0.86104645919428469, 0.82507184046967996, 0.85385450564596777)
(0.83039059245682523, 0.82318304141652632, 0.82894476104773207)
(0.83222389229268878, 0.84593146016740117, 0.83496609071694428)
(0.85769822822451824, 0.83669988224438052, 0.85352193817454669)
('results: ', 50, ' -- ', 0.81269680625857832, 0.84593146016740117, 0.82871660611131337)
(0.82733447416914574, 0.81358111915746611, 0.82458376213440876)
(0.85998455910990956, 0.82430600092867645, 0.85286227631919442)
(0.82385337019831295, 0.81647925396830456, 0.82237803274614718)
(0.85563373163944945, 0.85193541285457441, 0.85489688293370025)
(0.85948815896567154, 0.83640634781869516, 0.85489201316076291)
('results: ', 75, ' -- ', 0.81358111915746611, 0.85193541285457441, 0.82854162694554334)
(0.86151882117940148, 0.83478608705306312, 0.85617356255048715)
(0.86176085492922494, 0.82525998730975725, 0.85446189541780371)
(0.82026233281146843, 0.81221392099839773, 0.818650520

In [22]:
print (sum(y_fit==1)*1.0/len(y_fit))
print (len(y_fit), sum(y_fit==1))
print (sum(y_eval==1)*1.0/len(y_eval))
print (len(y_eval), sum(y_eval==1))

0.0389634306761
(38010, 1481)
0.0401736385162
(38010, 1527)


In [23]:
X_train.max(axis=1)

array([  39205.17      ,   49278.03      ,   67333.77      , ...,
         74028.15      ,   84278.16      ,  117310.97901649])

In [24]:
X_train.min(axis=1)

array([ 0., -1.,  0., ...,  0.,  0.,  0.])

In [25]:
%matplotlib inline  
import matplotlib.pyplot as plt

data_plt = X_train[:,28]
data_plt = data_plt[data_plt>=0]            
data_plt = np.log(data_plt+1)
#data_plt = data_plt[data_plt<25]
plt = hist(data_plt,10)

for j in unique(data_plt):
    print round(j,2), sum(data_plt==j), sum(y_train[data_plt==j])

NameError: name 'hist' is not defined